In [1]:
import hashlib
import copy
import queue

## Part 1

In [162]:
class Room():
    
    def __init__(self,pos_x, pos_y):
        self.pos_x = pos_x
        self.pos_y = pos_y
        if self.pos_y == 0:
            self.updoor = Door(False)
        else:
            self.updoor = Door(True)
        if self.pos_y == 3:
            self.downdoor = Door(False)
        else:
            self.downdoor = Door(True)
        if self.pos_x == 0:
            self.leftdoor = Door(False)
        else:
            self.leftdoor = Door(True)
        if self.pos_x == 3:
            self.rightdoor = Door(False)
        else:
            self.rightdoor = Door(True)
            
    def check_doors(self, code):
        if code[0] in ('b','c','d','e','f') and self.updoor.return_isdoor():
            self.updoor.open_door()
        if code[1] in ('b','c','d','e','f') and self.downdoor.return_isdoor():
            self.downdoor.open_door()
        if code[2] in ('b','c','d','e','f') and self.leftdoor.return_isdoor():
            self.leftdoor.open_door()
        if code[3] in ('b','c','d','e','f') and self.rightdoor.return_isdoor():
            self.rightdoor.open_door()

    def where_can_i_go(self):
        steps = []
        if self.updoor.return_isopen():
            steps.append('U')
        if self.downdoor.return_isopen():
            steps.append('D')
        if self.leftdoor.return_isopen():
            steps.append('L')
        if self.rightdoor.return_isopen():
            steps.append('R')
        return steps
            
    def is_vault(self):
        if self.pos_x == 3 and self.pos_y == 3:
            return True
        return False
    
    def get_pos(self):
        return (self.pos_x, self.pos_y)
    
    
        
class Door():
    
    def __init__(self, isdoor):
        self.isdoor = isdoor
        self.isopen = False
        
    def open_door(self):
        if not self.isdoor:
            return 'Error'
        self.isopen = True
        
    def return_isopen(self):
        if self.isopen:
            return True
        return False
    
    def return_isdoor(self):
        return self.isdoor
        
        
class Route():
    
    def __init__(self):
        self.steps = ''
        
    def add_step(self, step):
        self.steps += step
        
    def print_route(self):
        print (self.steps)
        
    def route_len(self):
        return len(self.steps)
        
    def return_code(self):
        m = hashlib.md5()
        m.update(b"rrrbmfta")
        m.update(str.encode(self.steps))
        code = m.hexdigest()[:4]
        
        return code
    
class Node():
    def __init__(self, route, room):
        self.route = route
        self.room = room
        
    def evaluate(self):
        if self.room.is_vault():
            return True
        return False
    
    def goal_estimate(self):
        return abs(3-self.room.get_pos()[0]) + abs(3-self.room.get_pos()[1])
    
    def nexts(self):        
        code = self.route.return_code()
        self.room.check_doors(code)
        directions = self.room.where_can_i_go()
        for d in directions:
            if d == 'D' and self.room.get_pos()[1] < 3:
                new_route = copy.copy(self.route)
                new_route.add_step('D')
                new_room = Room(self.room.get_pos()[0],self.room.get_pos()[1]+1)
                yield (Node(new_route,new_room))
            if d == 'R' and self.room.get_pos()[0] < 3:
                new_route = copy.copy(self.route)
                new_route.add_step('R')
                new_room = Room(self.room.get_pos()[0]+1,self.room.get_pos()[1])
                yield (Node(new_route,new_room))
            if d == 'U' and self.room.get_pos()[1] > 0:
                new_route = copy.copy(self.route)
                new_route.add_step('U')
                new_room = Room(self.room.get_pos()[0],self.room.get_pos()[1]-1)
                yield (Node(new_route,new_room))
            if d == 'L' and self.room.get_pos()[0] > 0:
                new_route = copy.copy(self.route)
                new_route.add_step('L')
                new_room = Room(self.room.get_pos()[0]-1,self.room.get_pos()[1])
                yield (Node(new_route,new_room))
                

In [91]:
def search(starting_node):
    q = queue.PriorityQueue()
    q.put((0 + starting_node.goal_estimate(), 0,0, starting_node))
    fake_iter = 1
    while not q.empty():
        (estimated_cost, spent_cost,fake_iter_nouse, node) = q.get()
        for n in node.nexts():
            if n.evaluate():
                print ('jee', n.route.print_route())
                return n
            new_cost = spent_cost + 1
            q.put((new_cost + n.goal_estimate(), new_cost, fake_iter, n))

            fake_iter += 1

In [92]:
n = search(Node(Route(), Room(0,0)))

RLRDRDUDDR
jee None


## Part 2

In [163]:
def search(starting_node):
    q = queue.Queue()
    q.put(starting_node)
    routes = []
    while not q.empty():
        node = q.get()
        for n in node.nexts():
            if n.evaluate():
                routes.append(n.route.route_len()) 
            else:                
                q.put(n)
    return max(routes)

In [164]:
search(Node(Route(), Room(0,0)))

420